In [1]:
import pandas as pd 
import pyarrow

In [2]:
import numpy as np


In [8]:
import sys
!{sys.executable} -m pip show pyarrow


Name: pyarrow
Version: 22.0.0
Summary: Python library for Apache Arrow
Home-page: https://arrow.apache.org/
Author: 
Author-email: 
License: Apache Software License
Location: /usr/local/Cellar/jupyterlab/4.3.5/libexec/lib/python3.13/site-packages
Requires: 
Required-by: 


In [9]:
import sys
!{sys.executable} -m pip install pyarrow


[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [10]:
!pip install pyarrow


[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [11]:
path_test_identity = '/Users/prasanna/Desktop/DataTalksClubs/capstone-project1/ieee-fraud-detection/test_identity.csv' 
path_train_identity = '/Users/prasanna/Desktop/DataTalksClubs/capstone-project1/ieee-fraud-detection/train_identity.csv' 
path_test_trans = '/Users/prasanna/Desktop/DataTalksClubs/capstone-project1/ieee-fraud-detection/test_transaction.csv'
path_train_trans='/Users/prasanna/Desktop/DataTalksClubs/capstone-project1/ieee-fraud-detection/train_transaction.csv'

In [12]:
import pyarrow

In [13]:
df = pd.read_csv(path_test_trans) 
# or use inferred dtypes
parquet_file = '/Users/prasanna/Desktop/DataTalksClubs/ml-projects-work/Customer_fraud_detection/datasets/test_transaction.parquet'
#df.to_parquet(parquet_file, engine="pyarrow", index=False)


In [31]:
df2 = pd.read_csv(path_train_trans)
parquet_trans_file = '/Users/prasanna/Desktop/DataTalksClubs/ml-projects-work/Customer_fraud_detection/datasets/train_transaction.parquet'
#df.to_parquet(parquet_trans_file, engine="pyarrow", index=False)


In [ ]:
df = pd.read_csv(path_test_identity)
# or use inferred dtypes
parquet_file1 = '/Users/prasanna/Desktop/DataTalksClubs/ml-projects-work/Customer_fraud_detection/datasets/test_identity.parquet'
#df.to_parquet(parquet_file1, engine="pyarrow", index=False)


In [14]:
csv_count = len(pd.read_csv(path_test_trans))
parquet_count = len(pd.read_parquet(parquet_file))
print(csv_count,parquet_count)

506691 144233


## converting csv file to parquet without loosing any rows 

In [18]:
print(csv_rows,parquet_rows)

506691 206691


#### From the above observed found few records are missind while converting csv to parquet file

In [23]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

chunksize = 300_000

writer = None
rows_written = 0

for chunk in pd.read_csv(path_test_trans, chunksize=chunksize, on_bad_lines="warn"):
    table = pa.Table.from_pandas(chunk)

    if writer is None:
        writer = pq.ParquetWriter(parquet_file, table.schema)

    writer.write_table(table)
    rows_written += len(chunk)
    print(f"Chunk rows: {len(chunk)} | Total so far: {rows_written}")

if writer:
    writer.close()

# Validate
csv_rows = sum(1 for _ in open(path_test_trans)) - 1
parquet_rows = rows_written

print("\n Validation Summary")
print("CSV rows      :", csv_rows)
print("Parquet rows  :", parquet_rows)
print("Row loss      :", csv_rows - parquet_rows)
print("Final file    :", parquet_file)


Chunk rows: 300000 | Total so far: 300000
Chunk rows: 206691 | Total so far: 506691

 Validation Summary
CSV rows      : 506691
Parquet rows  : 506691
Row loss      : 0
Final file    : /Users/prasanna/Desktop/DataTalksClubs/ml-projects-work/Customer_fraud_detection/datasets/test_transaction.parquet


In [22]:
len(pd.read_parquet(parquet_file))

506691

In [27]:
def convert_parquet(csv_file,parquet_file,chunksize,writer=None,rows_written=0): 
   
    for chunk in pd.read_csv(csv_file, chunksize=chunksize, on_bad_lines="warn"):
        table = pa.Table.from_pandas(chunk)
    
        if writer is None:
            writer = pq.ParquetWriter(parquet_file, table.schema)
    
        writer.write_table(table)
        rows_written += len(chunk)
        print(f" Chunk rows: {len(chunk)} | Total so far: {rows_written}")

    if writer:
        writer.close()
    
    # Validate
    csv_rows = sum(1 for _ in open(csv_file)) - 1
    parquet_rows = rows_written
    
    print("\n Validation Summary")
    print("CSV rows      :", csv_rows)
    print("Parquet rows  :", parquet_rows)
    print("Row loss      :", csv_rows - parquet_rows)
    print("Final file    :", parquet_file)

In [28]:
convert_parquet(csv_file=path_test_trans,parquet_file=parquet_file,chunksize=300_000)


 Chunk rows: 300000 | Total so far: 300000
 Chunk rows: 206691 | Total so far: 506691

 Validation Summary
CSV rows      : 506691
Parquet rows  : 506691
Row loss      : 0
Final file    : /Users/prasanna/Desktop/DataTalksClubs/ml-projects-work/Customer_fraud_detection/datasets/test_transaction.parquet


In [32]:
convert_parquet(csv_file=path_train_trans,parquet_file=parquet_trans_file,chunksize=300_000)


 Chunk rows: 300000 | Total so far: 300000
 Chunk rows: 290540 | Total so far: 590540

 Validation Summary
CSV rows      : 590540
Parquet rows  : 590540
Row loss      : 0
Final file    : /Users/prasanna/Desktop/DataTalksClubs/ml-projects-work/Customer_fraud_detection/datasets/train_transaction.parquet


In [33]:
len(pd.read_parquet(parquet_trans_file))

590540

In [5]:
path_test_identity = '/Users/prasanna/Desktop/DataTalksClubs/capstone-project1/ieee-fraud-detection/test_identity.csv' 
path_train_identity = '/Users/prasanna/Desktop/DataTalksClubs/capstone-project1/ieee-fraud-detection/train_identity.csv' 
parquet_test_identity = '/Users/prasanna/Desktop/DataTalksClubs/ml-projects-work/Customer_fraud_detection/datasets/test_identity.parquet'
parquet_train_identity = '/Users/prasanna/Desktop/DataTalksClubs/ml-projects-work/Customer_fraud_detection/datasets/train_identity.parquet'

## Cleaning Identity dataset and removing unwanted data and converting to parqeut format

In [35]:
df_test_identity = pd.read_csv(path_test_identity)
df_test_identity.head()

,TransactionID,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663586,-45.0,280290.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
1,3663588,0.0,3579.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 67.0 for android,24.0,1280x720,match_status:2,T,F,T,T,mobile,LGLS676 Build/MXB48T
2,3663597,-5.0,185210.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,...,ie 11.0 for tablet,NaN,NaN,NaN,F,T,T,F,desktop,Trident/7.0
3,3663601,-45.0,252944.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
4,3663602,-95.0,328680.0,NaN,NaN,7.0,-33.0,NaN,NaN,NaN,...,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,SM-G9650 Build/R16NW


In [41]:
df_test_id = df_test_identity[['TransactionID']].copy()
df_test_id.head()

,TransactionID
0,3663586
1,3663588
2,3663597
3,3663601
4,3663602


In [43]:
df_test_id.columns

Index(['TransactionID'], dtype='object')

In [3]:
path_train_identity = '/Users/prasanna/Desktop/DataTalksClubs/capstone-project1/ieee-fraud-detection/train_identity.csv' 

df_test_identity = pd.read_csv(path_train_identity)
df_test_identity.columns.tolist()

['TransactionID',
 'id_01',
 'id_02',
 'id_03',
 'id_04',
 'id_05',
 'id_06',
 'id_07',
 'id_08',
 'id_09',
 'id_10',
 'id_11',
 'id_12',
 'id_13',
 'id_14',
 'id_15',
 'id_16',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_21',
 'id_22',
 'id_23',
 'id_24',
 'id_25',
 'id_26',
 'id_27',
 'id_28',
 'id_29',
 'id_30',
 'id_31',
 'id_32',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38',
 'DeviceType',
 'DeviceInfo']

In [4]:
path_test_identity = '/Users/prasanna/Desktop/DataTalksClubs/capstone-project1/ieee-fraud-detection/test_identity.csv' 

df_test_identity = pd.read_csv(path_test_identity)
df_test_identity.columns.tolist()

['TransactionID',
 'id-01',
 'id-02',
 'id-03',
 'id-04',
 'id-05',
 'id-06',
 'id-07',
 'id-08',
 'id-09',
 'id-10',
 'id-11',
 'id-12',
 'id-13',
 'id-14',
 'id-15',
 'id-16',
 'id-17',
 'id-18',
 'id-19',
 'id-20',
 'id-21',
 'id-22',
 'id-23',
 'id-24',
 'id-25',
 'id-26',
 'id-27',
 'id-28',
 'id-29',
 'id-30',
 'id-31',
 'id-32',
 'id-33',
 'id-34',
 'id-35',
 'id-36',
 'id-37',
 'id-38',
 'DeviceType',
 'DeviceInfo']

In [8]:
def clean_identity_dataset(csv_file, parquet_file): 
    df_test_identity = pd.read_csv(csv_file)
    df_test_identity.to_parquet(parquet_file, engine="pyarrow", index=False)
    csv_count = len(df_test_identity)
    parquet_count = len(df_test_identity)
    print("\n Validation Summary")
    print("CSV rows      :", csv_count)
    print("Parquet rows  :", parquet_count)
    print("Final file    :", parquet_file)

    


In [9]:
clean_identity_dataset(csv_file=path_test_identity, parquet_file=parquet_test_identity)


 Validation Summary
CSV rows      : 141907
Parquet rows  : 141907
Final file    : /Users/prasanna/Desktop/DataTalksClubs/ml-projects-work/Customer_fraud_detection/datasets/test_identity.parquet


In [10]:
clean_identity_dataset(csv_file=path_train_identity, parquet_file=parquet_train_identity)


 Validation Summary
CSV rows      : 144233
Parquet rows  : 144233
Final file    : /Users/prasanna/Desktop/DataTalksClubs/ml-projects-work/Customer_fraud_detection/datasets/train_identity.parquet


In [11]:
test = pd.read_parquet(parquet_test_identity)
test.head()

,TransactionID,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663586,-45.0,280290.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,chrome 67.0 for android,NaN,None,None,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
1,3663588,0.0,3579.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 67.0 for android,24.0,1280x720,match_status:2,T,F,T,T,mobile,LGLS676 Build/MXB48T
2,3663597,-5.0,185210.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,...,ie 11.0 for tablet,NaN,None,None,F,T,T,F,desktop,Trident/7.0
3,3663601,-45.0,252944.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 67.0 for android,NaN,None,None,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
4,3663602,-95.0,328680.0,NaN,NaN,7.0,-33.0,NaN,NaN,NaN,...,chrome 67.0 for android,NaN,None,None,F,F,T,F,mobile,SM-G9650 Build/R16NW


In [50]:
train = pd.read_parquet(parquet_train_identity)
train.head()

,TransactionID
0,2987004
1,2987008
2,2987010
3,2987011
4,2987016
